# Energy system optimisation with oemof - how to collect and store results

### Import necessary modules

In [1]:
import os
import pandas as pd
from oemof.solph import (Sink, Source, Transformer, Bus, Flow, Model,
                         EnergySystem, processing, views)

import pickle

### Specify solver

In [13]:
solver = 'cbc'

### Create an energy system and optimize the dispatch at least costs.

In [14]:
# initialize and provide data
datetimeindex = pd.date_range('1/1/2016', periods=24*10, freq='H')
energysystem = EnergySystem(timeindex=datetimeindex)
filename = 'input_data.csv'
filename = os.path.join(os.getcwd(), filename)
data = pd.read_csv(filename, sep=",")

### Create and add components to energysystem

In [4]:
# resource buses
bcoal = Bus(label='coal', balanced=False)
bgas = Bus(label='gas', balanced=False)
boil = Bus(label='oil', balanced=False)
blig = Bus(label='lignite', balanced=False)

# electricity and heat
bel = Bus(label='bel')
bth = Bus(label='bth')

energysystem.add(bcoal, bgas, boil, blig, bel, bth)

In [5]:
# an excess and a shortage variable can help to avoid infeasible problems
energysystem.add(Sink(label='excess_el', inputs={bel: Flow()}))
# shortage_el = Source(label='shortage_el',
#                      outputs={bel: Flow(variable_costs=200)})

# sources
energysystem.add(Source(label='wind', outputs={bel: Flow(
    fix=data['wind'], nominal_value=66.3)}))

energysystem.add(Source(label='pv', outputs={bel: Flow(
    fix=data['pv'], nominal_value=65.3)}))

# demands (electricity/heat)
energysystem.add(Sink(label='demand_el', inputs={bel: Flow(
    nominal_value=85, fix=data['demand_el'])}))

energysystem.add(Sink(label='demand_th',
                 inputs={bth: Flow(nominal_value=40,
                                   fix=data['demand_th'],
                                   fixed=True)}))

/home/uwe/git-projects/oemof-solph/src/oemof/solph/network.py:184: SuspiciousUsageWarning: The `fixed` attribute is deprecated.
If you have defined the `fix` attribute the flow variable will be fixed.
The `fixed` attribute does not change anything.
  warn(msg, debugging.SuspiciousUsageWarning)


In [6]:
# power plants
energysystem.add(Transformer(
    label='pp_coal',
    inputs={bcoal: Flow()},
    outputs={bel: Flow(nominal_value=20.2, variable_costs=25)},
    conversion_factors={bel: 0.39}))

energysystem.add(Transformer(
    label='pp_lig',
    inputs={blig: Flow()},
    outputs={bel: Flow(nominal_value=11.8, variable_costs=19)},
    conversion_factors={bel: 0.41}))

energysystem.add(Transformer(
    label='pp_gas',
    inputs={bgas: Flow()},
    outputs={bel: Flow(nominal_value=41, variable_costs=40)},
    conversion_factors={bel: 0.50}))

energysystem.add(Transformer(
    label='pp_oil',
    inputs={boil: Flow()},
    outputs={bel: Flow(nominal_value=5, variable_costs=50)},
    conversion_factors={bel: 0.28}))

In [7]:
# combined heat and power plant (chp)
energysystem.add(Transformer(
    label='pp_chp',
    inputs={bgas: Flow()},
    outputs={bel: Flow(nominal_value=30, variable_costs=42),
             bth: Flow(nominal_value=40)},
    conversion_factors={bel: 0.3, bth: 0.4}))

In [8]:
# heat pump with a coefficient of performance (COP) of 3
b_heat_source = Bus(label='b_heat_source')
energysystem.add(b_heat_source)

energysystem.add(Source(label='heat_source', outputs={b_heat_source: Flow()}))

cop = 3
energysystem.add(Transformer(
    label='heat_pump',
    inputs={bel: Flow(),
            b_heat_source: Flow()},
    outputs={bth: Flow(nominal_value=10)},
    conversion_factors={bel: 1/3, b_heat_source: (cop-1)/cop}))

### Optimization

In [9]:
# create optimization model based on energy_system
optimization_model = Model(energysystem=energysystem)

# solve problem
optimization_model.solve(solver=solver,
                         solve_kwargs={'tee': True, 'keepfiles': False})

Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Aug 21 2017 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmpzvc245k8.pyomo.lp -stat=1 -solve -solu /tmp/tmpzvc245k8.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 224 (-2417) rows, 1120 (-2721) columns and 1120 (-5361) elements
Statistics for presolved model


Problem has 224 rows, 1120 columns (1120 with objective) and 1120 elements
There are 1120 singletons with objective 
Column breakdown:
0 of type 0.0->inf, 997 of type 0.0->up, 0 of type lo->inf, 
123 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
0 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
224 of type G other, 0 of type L 0.0, 0 of type L 1.0, 
0 of type L other, 0 of type Range 0.0->1.0, 0 of type Range other, 
0 of type Free 
Presolve 224 (-2417) rows, 1120 (-2721

{'Problem': [{'Name': 'unknown', 'Lower bound': 383538.2988, 'Upper bound': 383538.2988, 'Number of objectives': 1, 'Number of constraints': 2641, 'Number of variables': 3841, 'Number of nonzeros': 1120, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.01, 'Wallclock time': 0.02, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}, 'Black box': {'Number of iterations': 224}}, 'Error rc': 0, 'Time': 0.05781960487365723}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

### Write results into energysystem.results object for later

In [10]:
energysystem.results['main'] = processing.results(optimization_model)
energysystem.results['meta'] = processing.meta_results(optimization_model)

In [11]:
string_results = views.convert_keys_to_strings(energysystem.results['main'])

### Save results - Dump the energysystem (to ~/home/user/.oemof by default)
Specify path and filename if you do not want to overwrite

In [12]:
energysystem.dump(dpath=None, filename=None)

'Attributes dumped to: /home/uwe/.oemof/dumps/es_dump.oemof'